In [53]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.optimizers import RMSprop
import random

In [10]:
filepath=tf.keras.utils.get_file('shakspear.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [27]:
text=open(filepath,'r').read().lower()

In [28]:
text=text[20000:80000]

In [29]:
characters=sorted(set(text))

In [30]:
text

"the good horse is mine.\n\nmarcius:\ni'll buy him of you.\n\nlartius:\nno, i'll nor sell nor give him: lend you him i will\nfor half a hundred years. summon the town.\n\nmarcius:\nhow far off lie these armies?\n\nmessenger:\nwithin this mile and half.\n\nmarcius:\nthen shall we hear their 'larum, and they ours.\nnow, mars, i prithee, make us quick in work,\nthat we with smoking swords may march from hence,\nto help our fielded friends! come, blow thy blast.\ntutus aufidius, is he within your walls?\n\nfirst senator:\nno, nor a man that fears you less than he,\nthat's lesser than a little.\nhark! our drums\nare bringing forth our youth. we'll break our walls,\nrather than they shall pound us up: our gates,\nwhich yet seem shut, we, have but pinn'd with rushes;\nthey'll open of themselves.\nhark you. far off!\nthere is aufidius; list, what work he makes\namongst your cloven army.\n\nmarcius:\no, they are at it!\n\nlartius:\ntheir noise be our instruction. ladders, ho!\n\nmarcius:\nthey 

In [31]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [32]:
char_to_index
print(index_to_char)

{0: '\n', 1: ' ', 2: '!', 3: '&', 4: "'", 5: ',', 6: '-', 7: '.', 8: ':', 9: ';', 10: '?', 11: 'a', 12: 'b', 13: 'c', 14: 'd', 15: 'e', 16: 'f', 17: 'g', 18: 'h', 19: 'i', 20: 'j', 21: 'k', 22: 'l', 23: 'm', 24: 'n', 25: 'o', 26: 'p', 27: 'q', 28: 'r', 29: 's', 30: 't', 31: 'u', 32: 'v', 33: 'w', 34: 'x', 35: 'y', 36: 'z'}


In [36]:
sequence_lenth=40
step_size=3
sentences=[]
next_char=[]
for i in range(0,len(text)-sequence_lenth,step_size):
    sentences.append(text[i:i+sequence_lenth])
    next_char.append(text[i+sequence_lenth])


In [38]:
x=np.zeros((len(sentences),sequence_lenth,len(characters)),dtype=bool)
y=np.zeros((len(sentences),len(characters)),dtype=bool)

In [40]:
x[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [41]:
for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_to_index[char]]=1
    y[i,char_to_index[next_char[i]]]=1

In [47]:
model=Sequential()
model.add(LSTM(128,input_shape=(sequence_lenth,len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [50]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01))
model.fit(x,y,batch_size=256,epochs=4)

Epoch 1/4
79/79 ━━━━━━━━━━━━━━━━━━━━ 14s 161ms/step - loss: 2.8026
Epoch 2/4
79/79 ━━━━━━━━━━━━━━━━━━━━ 20s 151ms/step - loss: 2.2566
Epoch 3/4
79/79 ━━━━━━━━━━━━━━━━━━━━ 20s 152ms/step - loss: 2.0594
Epoch 4/4
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 138ms/step - loss: 1.9030


In [51]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [54]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - sequence_lenth - 1)
    generated = ''
    sentence = text[start_index: start_index + sequence_lenth]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, sequence_lenth, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [56]:
print(generate_text(300, 0.2))
print("0.4")
print(generate_text(300, 0.4))
print("0.6")
print(generate_text(300, 0.6))
print("0.8")
print(generate_text(300, 0.8))
print("1")
print(generate_text(300, 1))

 to issue out of
one skull, they would for he heme to but heme heme home he the the the whe heme heme heme he stond to menger mong and woun and to menger mong and to sugher mong and to mengon to cons and to menger to ent to men to menger to menger mong and to me cons and to and to suge to not not he the the butt and mong to the mongon and
0.4
 coming home!

volumnia:
ay, worthy mene to mengen so pomse he when your thengere wo home heme mond and to and a gogd a dome here to sugh mongan so she munt to pent to meng on the gong to gond mong ang stond to neme he woung to men not not me to mend of the shing the doman as to the gont
to not of shome heme so me to mong to whe to she po
0.6
sty plebeians, hate thine honours,
shall mencesencond foure to menger to your goiten a duge go so whe cons long of coming here hong come be to nom not not mong the ars and woun a to be they enot' tring in there to sengom oup erest but he oft ou not heme romy eng and nom wer mome and to gome fot heme you hang 